# I will try to finetune a small multilingual model  while using the constructed triplets :

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import random
from tqdm import tqdm

import os
from torch.utils.tensorboard.writer import SummaryWriter
import numpy as np
import time
from datasets import Dataset

import sentence_transformers
from sentence_transformers import SentenceTransformer
import torch



In [10]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_triplets.reset_index(drop=True)) # train_triplets is private
eval_dataset = Dataset.from_pandas(valid_triplets.reset_index(drop=True))  # valid_triplets is private

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps')

torch.cuda.empty_cache()

link = "all-MiniLM-L6-v2"
models = SentenceTransformer(link).to(device = device, dtype = dtype)



In [14]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
# 3. Define a loss function
loss = MultipleNegativesRankingLoss(models)

In [15]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/mpnet-base-all-nli-triplet",
    # Optional training parameters:
    num_train_epochs= 3,
    per_device_train_batch_size = 8, #2
    per_device_eval_batch_size= 8,  #2
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="mpnet-base-all-nli-triplet",  # Will be used in W&B if `wandb` is installed
)

In [16]:
trainer = SentenceTransformerTrainer(
    model=models,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss
)

In [17]:
trainer.train()

  0%|          | 0/2097 [00:00<?, ?it/s]

{'loss': 1.5785, 'grad_norm': 20.825035095214844, 'learning_rate': 2.380952380952381e-05, 'epoch': 0.14}


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

{'loss': 1.4362, 'grad_norm': 11.926809310913086, 'learning_rate': 4.761904761904762e-05, 'epoch': 0.29}
{'loss': 1.567, 'grad_norm': 16.931535720825195, 'learning_rate': 4.7615262321144675e-05, 'epoch': 0.43}
{'loss': 1.5416, 'grad_norm': 8.389205932617188, 'learning_rate': 4.49655537890832e-05, 'epoch': 0.57}
{'loss': 1.4882, 'grad_norm': 11.200145721435547, 'learning_rate': 4.231584525702173e-05, 'epoch': 0.72}
{'loss': 1.4042, 'grad_norm': 13.053640365600586, 'learning_rate': 3.9666136724960254e-05, 'epoch': 0.86}
{'loss': 1.3641, 'grad_norm': 7.60820198059082, 'learning_rate': 3.701642819289878e-05, 'epoch': 1.0}
{'loss': 1.3309, 'grad_norm': 8.626004219055176, 'learning_rate': 3.436671966083731e-05, 'epoch': 1.14}
{'loss': 1.0378, 'grad_norm': 8.600197792053223, 'learning_rate': 3.171701112877583e-05, 'epoch': 1.28}
{'loss': 1.0371, 'grad_norm': 14.462275505065918, 'learning_rate': 2.9067302596714363e-05, 'epoch': 1.43}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 1.009225845336914, 'eval_runtime': 3.6662, 'eval_samples_per_second': 15.002, 'eval_steps_per_second': 1.909, 'epoch': 1.43}
{'loss': 1.0746, 'grad_norm': 7.453387260437012, 'learning_rate': 2.641759406465289e-05, 'epoch': 1.57}
{'loss': 1.0126, 'grad_norm': 22.081600189208984, 'learning_rate': 2.3767885532591416e-05, 'epoch': 1.71}
{'loss': 1.0417, 'grad_norm': 9.397717475891113, 'learning_rate': 2.1118177000529942e-05, 'epoch': 1.86}
{'loss': 0.9662, 'grad_norm': 7.149302005767822, 'learning_rate': 1.846846846846847e-05, 'epoch': 2.0}
{'loss': 0.9587, 'grad_norm': 6.786861419677734, 'learning_rate': 1.5818759936406995e-05, 'epoch': 2.14}
{'loss': 0.6763, 'grad_norm': 11.285944938659668, 'learning_rate': 1.3169051404345523e-05, 'epoch': 2.28}
{'loss': 0.7274, 'grad_norm': 12.25250244140625, 'learning_rate': 1.0519342872284049e-05, 'epoch': 2.42}
{'loss': 0.7977, 'grad_norm': 14.143073081970215, 'learning_rate': 7.869634340222575e-06, 'epoch': 2.57}
{'loss': 0.8116, 'grad

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.8319634795188904, 'eval_runtime': 1.5317, 'eval_samples_per_second': 35.909, 'eval_steps_per_second': 4.57, 'epoch': 2.85}
{'train_runtime': 985.6895, 'train_samples_per_second': 17.02, 'train_steps_per_second': 2.127, 'train_loss': 1.127007174730642, 'epoch': 2.99}


TrainOutput(global_step=2097, training_loss=1.127007174730642, metrics={'train_runtime': 985.6895, 'train_samples_per_second': 17.02, 'train_steps_per_second': 2.127, 'total_flos': 0.0, 'train_loss': 1.127007174730642, 'epoch': 2.991416309012876})

In [ ]:
from sentence_transformers.evaluation import TripletEvaluator


test_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    name="all-nli-test",
)

test_evaluator(models)
